In [1]:
import os
import pandas as pd
import pprint as pp
from pathlib import Path

from utils.model import Model

# Load Dataset

In [2]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])

In [3]:
dataset_dir = Path(base_dir).joinpath(f"data/dataset/{DOMAIN}/interactions")
df = pd.read_parquet(dataset_dir)
df.head()

,user_id,item_id,timestamp,action,age,gender,title,color,style,fit,material,season,sleeve,category
0,1,3092,2025-09-21 03:01:00,click,46,F,"트렌디한 스포티룩, 세미오버핏 레깅스 in 여름",아이보리,스포티,세미오버핏,울,여름,7부,레깅스
1,1,8879,2025-09-21 03:02:00,click,46,F,"트렌디한 스포티룩, 슬림핏 레깅스 in 여름",아이보리,스포티,슬림핏,메쉬,여름,롱슬리브,레깅스
2,1,1880,2025-09-21 03:03:00,click,46,F,필수템! 스포티 무드의 간절기용 레깅스,아이보리,스포티,세미오버핏,데님,간절기,숏슬리브,레깅스
3,1,6154,2025-09-21 03:13:00,click,46,F,여름 감성 슬림핏 폴리 레깅스,아이보리,스포티,슬림핏,폴리,여름,롱슬리브,레깅스
4,1,8642,2025-09-21 03:13:00,wishlist,46,F,"트렌디한 스포티룩, 레귤러핏 레깅스 in 겨울",아이보리,스포티,레귤러핏,코튼,겨울,7부,레깅스


# Load Model

In [4]:
DOMAIN = "fashion"

current_dir = os.path.abspath(os.curdir)
base_dir = "/".join(current_dir.split("/")[:-1])
model_dir = Path(base_dir).joinpath(f"data/model/{DOMAIN}")

In [5]:
model = Model(model_dir=model_dir)

/Users/hwangyungwan/miniforge3/envs/ds_p313/lib/python3.13/site-packages/torch/nn/modules/transformer.py:382: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


## Prediction Test

In [6]:
body = [
    {
        "user_id": 123,
        "inputs": [
            {
                "color": "화이트",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "코튼",
                "season": "가을",
                "sleeve": "롱",
                "category": "블라우스",
            },
            {
                "color": "그레이",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
        ],
    }
]

pp.pprint(model.predict(body))

[{'item_vector': [0.012756336480379105,
                  0.018803928047418594,
                  -0.015087392181158066,
                  0.0214197039604187,
                  0.020925557240843773,
                  -0.02413874864578247,
                  -0.022991187870502472,
                  -0.03515581041574478,
                  0.024988405406475067,
                  0.02877390943467617,
                  -0.025987371802330017,
                  -0.003521174192428589,
                  0.17146672308444977,
                  0.025886476039886475,
                  -0.036817874759435654,
                  -0.0285935178399086,
                  0.0007115602493286133,
                  -0.02990095317363739,
                  0.07703650742769241,
                  0.010875731706619263,
                  0.038225527852773666,
                  -0.008665965870022774,
                  0.052017830312252045,
                  -0.02618199959397316,
                  0.024162620306015015,

## Serving Test

In [13]:
# CLI 실행 후 테스트 필요
# `bentoml serve service:svc --reload`

import requests

url = "http://localhost:3000/predict"
payload = [
    {
        "user_id": 123,
        "inputs": [
            {
                "color": "화이트",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "코튼",
                "season": "가을",
                "sleeve": "롱",
                "category": "블라우스",
            },
            {
                "color": "블랙",
                "style": "포멀",
                "fit": "오버핏",
                "material": "퍼",
                "season": "가을",
                "sleeve": "롱",
                "category": "점퍼",
            },
            {
                "color": "블랙",
                "style": "캐주얼",
                "fit": "레귤러핏",
                "material": "가죽",
                "season": "겨울",
                "sleeve": "롱",
                "category": "조끼",
            },
        ],
    }
]

response = requests.post(url, json=payload)
response = response.json()
pp.pprint(response)

{'predictions': [{'outputs': {'category': ['블라우스', '셔츠', '점퍼', '조끼'],
                              'color': ['그레이', '블랙', '화이트'],
                              'fit': ['레귤러핏', '루즈핏', '오버핏'],
                              'material': ['가죽', '코튼', '퍼'],
                              'season': ['가을', '겨울', '여름'],
                              'sleeve': ['7부', '롱'],
                              'style': ['레트로', '캐주얼', '포멀']},
                  'user_id': 123,
                  'user_vector': [0.16392560303211212,
                                  -0.1345544457435608,
                                  -0.037076838314533234,
                                  -0.07612133026123047,
                                  -0.11880791187286377,
                                  0.027564458549022675,
                                  -0.14626041054725647,
                                  0.02287314645946026,
                                  0.25322389602661133,
                                  -0.

# Recommendation

In [14]:
item_metadata_path = Path(base_dir).joinpath(
    f"data/dataset/{DOMAIN}/item_metadata.parquet"
)
df_item = pd.read_parquet(item_metadata_path)

In [15]:
response["predictions"][0]["outputs"]

{'color': ['그레이', '블랙', '화이트'],
 'style': ['레트로', '캐주얼', '포멀'],
 'fit': ['레귤러핏', '루즈핏', '오버핏'],
 'material': ['가죽', '코튼', '퍼'],
 'season': ['가을', '겨울', '여름'],
 'sleeve': ['7부', '롱'],
 'category': ['블라우스', '셔츠', '점퍼', '조끼']}